In [2]:
import pandas as pd
import sklearn
import numpy as np


#### Cleaning the Data file - Dropping Features and Converting Categorical to Numerical


In [3]:
data = pd.read_csv("data/out.csv")

# Modify the 'winner' column based on conditions
data['winner'] = data.apply(lambda row: 1 if row['winner'] == row['fighter_1'] else 0, axis=1)


# Drop the specified columns
columns_to_drop = ['fight_id', 'fighter_1_id', 'fighter_1', 'fighter_2', 'fighter_2_id','decision_method','fight_duration_lastrnd','time_format','date', 'location','height_x','name_x','weight_x','height_y','name_y','weight_y' ]
data_cleaned = data.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned data
data_cleaned.head()

data_cleaned.to_csv("data/cleanedData2.csv", index = False)

In [4]:
df = pd.read_csv("data/cleanedDataNEW_encoded.csv") # original dataset
##print(f"Initial shape: {df.shape}")

'''keep = np.array([4,2,1,6,10,11,12,37,38,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,104,105,130,132,133,134,135,136,137,138,139,140,141,142,143,144,145]) - 1
df = df.iloc[:,:] # interesting fields

df["date"] = pd.to_datetime(df["date"]) # date as datetime


df = df.replace(r'^\s*$', np.nan, regex=True) # replace empty string with nan
df = df.fillna(np.nan) # Fill empty and NaNs values with NaN

df = df.round(3) # format data

df.rename(columns={ # some renaming
    "B_win_by_KO/TKO": "B_win_by_KO_TKO",
    "R_win_by_KO/TKO": "R_win_by_KO_TKO"},inplace=True)
##print(f"shape: {df.shape}")

# considering matches with a winner as this will be a binary classification
drawRate = round(len(df["Winner"][df["Winner"]== "Draw"])/len(df.index)*100,3)
print(f"Draw rate is: {drawRate}%, converting to nan")
df["Winner"] = df["Winner"].replace("Draw", np.nan) # convert to nan to be dropped later
##df.dropna()'''



df

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_882688/1623933302.py:4: SyntaxWarning: invalid escape sequence '\s'
  '''keep = np.array([4,2,1,6,10,11,12,37,38,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,104,105,130,132,133,134,135,136,137,138,139,140,141,142,143,144,145]) - 1


,winner,decision_method,fight_duration_lastrnd,fight_duration_lastrnd_time,time_format,weight_class,date,location,dob_x,height_x,...,win_percentage_x,win_percentage_y,reach_advantage,total_fights_x,total_fights_y,finishing_rate_x,finishing_rate_y,age_x,age_y,age_difference
0,0,3,3,292,1,2,"November 02, 2019",92,835,"5' 7""",...,0.800000,0.846154,-3.0,20.0,13.0,0.242500,0.406154,30,28,2
1,1,7,3,164,1,0,"September 28, 2019",37,256,"5' 8""",...,1.000000,0.714286,0.0,12.0,14.0,0.398333,0.094286,24,25,-1
2,0,4,3,292,1,58,"November 16, 2019",126,612,"5' 5""",...,0.588235,0.875000,0.0,17.0,8.0,0.211765,0.603750,31,25,6
3,1,7,3,173,1,0,"October 05, 2019",78,286,"5' 5""",...,0.875000,0.687500,4.0,16.0,16.0,0.196250,0.096250,27,29,-2
4,0,4,3,292,1,4,"October 18, 2019",23,466,"6' 7""",...,0.666667,0.739130,7.0,9.0,23.0,0.288889,0.269565,29,28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3822,1,4,3,292,1,3,"September 21, 2019",80,126,"5' 6""",...,0.782609,0.620690,1.0,23.0,29.0,0.186087,0.092069,26,35,-9
3823,0,8,3,109,1,61,"September 21, 2019",80,737,"5' 2""",...,0.857143,0.588235,-3.0,14.0,17.0,0.430714,0.351765,26,34,-8
3824,1,4,3,292,1,58,"September 21, 2019",80,606,"5' 9""",...,0.687500,0.588235,3.0,16.0,17.0,0.393750,0.211765,31,31,0
3825,0,5,2,84,1,2,"September 21, 2019",80,901,"5' 8""",...,0.800000,0.666667,1.0,15.0,27.0,0.510667,0.187037,26,29,-3


In [5]:
##shuffling the data 
df_shuffled = df.sample(frac = 1)
df_shuffled.head(n=5)



,winner,decision_method,fight_duration_lastrnd,fight_duration_lastrnd_time,time_format,weight_class,date,location,dob_x,height_x,...,win_percentage_x,win_percentage_y,reach_advantage,total_fights_x,total_fights_y,finishing_rate_x,finishing_rate_y,age_x,age_y,age_difference
3253,1,5,2,56,1,7,"February 03, 2018",15,342,"6' 2""",...,0.750000,0.695652,0.0,28.0,46.0,0.162143,0.083913,34,29,5
2732,1,7,2,222,1,7,"June 03, 2017",112,578,"6' 2""",...,0.733333,0.666667,6.0,15.0,18.0,0.187333,0.261667,27,30,-3
1814,1,4,3,292,1,2,"November 08, 2014",141,720,"5' 8""",...,0.777778,0.571429,3.0,9.0,21.0,0.252222,0.053810,22,27,-5
3389,1,7,3,154,1,2,"May 27, 2018",71,368,"5' 8""",...,0.937500,0.750000,1.0,16.0,12.0,0.219375,0.140000,24,24,0
595,0,6,2,101,1,6,"June 26, 2011",106,112,"5' 8""",...,0.714286,0.777778,-6.0,42.0,36.0,0.113571,0.166944,26,21,5


In [6]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# Load the dataset
file_path = 'data/cleanedDataNEW.csv'
data = pd.read_csv(file_path)

# Specify the columns to encode
columns_to_encode = ['decision_method', 'time_format', 'weight_class',]

# Initialize the OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Apply ordinal encoding to the specified columns
data[columns_to_encode] = ordinal_encoder.fit_transform(data[columns_to_encode])

# Save the modified dataset
output_file_path = 'data/ordinal_encoded_cleanedDataNEW.csv'
data.to_csv(output_file_path, index=False)

print(f"Ordinal encoded dataset saved to {output_file_path}")



Ordinal encoded dataset saved to data/ordinal_encoded_cleanedDataNEW.csv


In [7]:
##split up the data 
df_shuffled.columns

'''x = df_shuffled.drop([], axis = 1).values
y = df_shuffled["Winner"].values'''
y = df_shuffled.iloc[:,0]
x = df_shuffled.iloc[:,1]

In [8]:
import numpy as np

class CustomMinMaxScaler:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    def fit(self, data):
        # Compute min and max values for each feature
        self.min_ = np.min(data, axis=0)
        self.max_ = np.max(data, axis=0)

    def transform(self, data):
        # Scale data to the range [0, 1]
        return (data - self.min_) / (self.max_ - self.min_)

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size = .2)

In [10]:
'''winners = y = df_shuffled["Winner"].values'''

scaler = CustomMinMaxScaler()

xTrainScaled = scaler.fit_transform(xTrain)
xTestScaled = scaler.fit_transform(xTest)
yTrainScaled = scaler.fit_transform(yTrain)
yTestScaled = scaler.fit_transform(yTest)


In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

# Define the parameters for GradientBoostingClassifier
n_estimators = 100      # Number of boosting stages (trees)
learning_rate = 0.2      # Step size at each iteration
max_depth = 3            # Maximum depth of the individual trees
subsample = 1.0          # Proportion of samples used to fit each tree
min_samples_split = 2    # Minimum number of samples required to split an internal node
min_samples_leaf = 1     # Minimum number of samples required to be at a leaf node
max_features = None      # Number of features to consider for the best split

# Assuming your data is preprocessed into X (features) and y (labels)

# Convert Pandas Series to NumPy arrays if necessary
xTrainScaled = xTrainScaled.to_numpy() if isinstance(xTrainScaled, pd.Series) else xTrainScaled
yTrainScaled = yTrainScaled.to_numpy() if isinstance(yTrain, pd.Series) else yTrain

# Debugging: Check the shape of the input data
print(f"xTrainScaled shape: {xTrainScaled.shape}, yTrain shape: {yTrain.shape}")

# Ensure that X is a 2D array (n_samples, n_features) and y is a 1D array (n_samples,)
if len(xTrainScaled.shape) == 1:  # If X has only one feature (1D array)
    xTrainScaled = xTrainScaled.reshape(-1, 1)  # Reshape it to be 2D (n_samples, 1)

if len(yTrain.shape) > 1:  # If y has more than one column
    yTrainScaled = yTrainScaled.reshape(-1, 1)  # Flatten it to be a 1D array

# Double-check the shape again after reshaping
print(f"xTrainScaled shape after reshaping: {xTrainScaled.shape}, yTrain shape after reshaping: {yTrain.shape}")

# Define the Gradient Boosting model with the specified parameters
model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    subsample=subsample,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    max_features=max_features,
    random_state=42
)

# Train the model on the training data
model.fit(xTrainScaled, yTrainScaled)

# Convert xTestScaled to numpy if it is a pandas Series, then reshape if it's 1D
if isinstance(xTestScaled, pd.Series):
    xTestScaled = xTestScaled.to_numpy()

# Reshape xTestScaled if it's 1D
if len(xTestScaled.shape) == 1:
    xTestScaled = xTestScaled.reshape(-1, 1)

# Make predictions using the trained model
y_pred = model.predict(xTestScaled)

# Calculate the accuracy of the model
accuracy = accuracy_score(yTest, y_pred)

# Calculate the precision of the model
precision = precision_score(yTest, y_pred, average='binary')  # Use 'binary' for binary classification tasks

# Print the final results
print(f"Test Set Accuracy: {accuracy:.4f}")
print(f"Test Set Precision: {precision:.4f}")


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [22]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd  # Ensure pandas is imported if using pandas DataFrames or Series

# Define the parameters for XGBClassifier
n_estimators = 400      # Number of boosting stages (trees)
learning_rate = 0.01    # Step size at each iteration
max_depth = 3           # Maximum depth of the individual trees
subsample = .8         # Proportion of samples used to fit each tree
min_child_weight = 1    # Minimum sum of instance weights (hessian) needed in a child
colsample_bytree = None # Proportion of features used for each tree (None defaults to 1.0)

# Assuming your data is preprocessed into X (features) and y (labels)

# Convert Pandas Series to NumPy arrays, if necessary
xTrainScaled = xTrainScaled.to_numpy() if isinstance(xTrainScaled, pd.Series) else xTrainScaled
yTrainScaled = yTrain.to_numpy() if isinstance(yTrain, pd.Series) else yTrain

# Debugging: Check the shape of the input data
print(f"xTrainScaled shape: {xTrainScaled.shape}, yTrain shape: {yTrain.shape}")

# Ensure that X is a 2D array (n_samples, n_features) and y is a 1D array (n_samples,)
if len(xTrainScaled.shape) == 1:  # If X has only one feature (1D array)
    xTrainScaled = xTrainScaled.reshape(-1, 1)  # Reshape it to be 2D (n_samples, 1)

if len(yTrainScaled.shape) > 1:  # If y has more than one column
    yTrainScaled = yTrainScaled.reshape(-1)  # Flatten it to be a 1D array

# Double-check the shape again after reshaping
print(f"xTrainScaled shape after reshaping: {xTrainScaled.shape}, yTrain shape after reshaping: {yTrainScaled.shape}")

# Define the XGBClassifier model with the specified parameters
model = XGBClassifier(
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    subsample=subsample,
    min_child_weight=min_child_weight,
    colsample_bytree=colsample_bytree if colsample_bytree else 1.0,
    random_state=42,
    use_label_encoder=False  # Add to suppress label encoder warning in XGBoost
)

# Train the model on the training data
model.fit(xTrainScaled, yTrainScaled)

# Convert xTestScaled to numpy if it is a pandas Series, then reshape if it's 1D
if isinstance(xTestScaled, pd.Series):
    xTestScaled = xTestScaled.to_numpy()

# Reshape xTestScaled if it's 1D
if len(xTestScaled.shape) == 1:
    xTestScaled = xTestScaled.reshape(-1, 1)

# Make predictions using the trained model
y_pred = model.predict(xTestScaled)

# Calculate the accuracy of the model
accuracy = accuracy_score(yTest, y_pred)

# Print the final results
print(f"Test Set Accuracy: {accuracy:.4f}")


xTrainScaled shape: (3061, 1), yTrain shape: (3061,)
xTrainScaled shape after reshaping: (3061, 1), yTrain shape after reshaping: (3061,)


/home/joshuat/UFC-Predictor/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [09:22:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Set Accuracy: 0.6123


In [23]:
import pandas as pd

# Load the dataset
file_path = 'data/cleanedDataNEW.csv'
data = pd.read_csv(file_path)

# Specify the columns to encode
columns_to_encode = ['decision_method', 'time_format', 'weight_class']

# Perform one-hot encoding
encoded_data = pd.get_dummies(data, columns=columns_to_encode, drop_first=True)

# Save the modified dataset
output_file_path = 'data/NEWencoded_cleanedDataNEW.csv'
encoded_data.to_csv(output_file_path, index=False)

print("done")


done


In [24]:

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Function to convert height strings to numeric (in inches)
def convert_height(height):
    try:
        parts = height.split("'")
        feet = int(parts[0])
        inches = int(parts[1].replace('"', '').strip())
        return feet * 12 + inches
    except:
        return np.nan  # Return NaN if conversion fails

# Load the dataset
file_path = "data/cleanedDataNEW_encoded.csv"  
data = pd.read_csv(file_path)

# Convert height columns to numeric
if 'height_x' in data.columns:
    data['height_x'] = data['height_x'].apply(convert_height)
if 'height_y' in data.columns:
    data['height_y'] = data['height_y'].apply(convert_height)

# Fill missing height values with the median height
for col in ['height_x', 'height_y']:
    if col in data.columns:
        data[col].fillna(data[col].median(), inplace=True)


# Define target column
target = 'winner'

# Handle categorical data
categorical_columns = ['decision_method', 'stance_x', 'stance_y', 'time_format', 'weight_class']
available_categorical = [col for col in categorical_columns if col in data.columns]
data_encoded = pd.get_dummies(data, columns=available_categorical, drop_first=True)

# Define the columns to exclude
exclude_columns = [
    'weight_x', 'weight_y', 'fight_duration_lastrnd_time', 'date', 'location',
    'dob_x', 'name_x', 'dob_y', 'name_y', target  
]

# Prepare features for training by excluding specific columns
existing_features = [col for col in data_encoded.columns if col not in exclude_columns]
print("Features used for training:", existing_features)

transformed_file_path = "data/transformedData.csv"
data_encoded.to_csv(transformed_file_path, index=False)
print(f"saved to {transformed_file_path}")
# Prepare data for training
X = data_encoded[existing_features]
y = data_encoded[target]

# Check for missing values and handle them
if X.isnull().sum().sum() > 0:
    print("imputing missing vals")
    X.fillna(X.median(), inplace=True)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting model
model = GradientBoostingClassifier(
    n_estimators=100,       # Number of trees
    learning_rate=0.1,      # Learning rate
    max_depth=3,            # Maximum tree depth
    min_samples_split=2,    # Stop splitting nodes with maximum instances
    subsample=1.0,          # Fraction of training instances
    random_state=42         # Replicable training
)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
# print("\nClassification Report:\n", classification_report(y_test, y_pred))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save the trained model to a file
joblib_file = "joblibs/gradient_boosting_model.joblib"  # Specify the file name
joblib.dump(model, joblib_file)

print(f"Model saved to {joblib_file}")



/tmp/ipykernel_882688/2181337754.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipykernel_882688/2181337754.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

Features used for training: ['fight_duration_lastrnd', 'height_x', 'n_draw_x', 'n_loss_x', 'n_win_x', 'reach_x', 'sig_str_abs_pM_x', 'sig_str_def_pct_x', 'sig_str_land_pM_x', 'sig_str_land_pct_x', 'sub_avg_x', 'td_avg_x', 'td_def_pct_x', 'td_land_pct_x', 'height_y', 'n_draw_y', 'n_loss_y', 'n_win_y', 'reach_y', 'sig_str_abs_pM_y', 'sig_str_def_pct_y', 'sig_str_land_pM_y', 'sig_str_land_pct_y', 'sub_avg_y', 'td_avg_y', 'td_def_pct_y', 'td_land_pct_y', 'win_percentage_x', 'win_percentage_y', 'reach_advantage', 'total_fights_x', 'total_fights_y', 'finishing_rate_x', 'finishing_rate_y', 'age_x', 'age_y', 'age_difference', 'decision_method_1', 'decision_method_2', 'decision_method_3', 'decision_method_4', 'decision_method_5', 'decision_method_6', 'decision_method_7', 'decision_method_8', 'stance_x_1', 'stance_x_2', 'stance_x_3', 'stance_y_1', 'stance_y_2', 'stance_y_3', 'time_format_1', 'time_format_2', 'time_format_3', 'weight_class_1', 'weight_class_2', 'weight_class_3', 'weight_class_4',

In [25]:
import numpy as np
import joblib 
import sklearn

print("Numpy version:", np.__version__)
print("Joblib version:", joblib.__version__)
print("scikit-learn", sklearn.__version__)


Numpy version: 2.1.3
Joblib version: 1.4.2
scikit-learn 1.5.2


### Loading in the Model to make predictions

In [27]:
import joblib
import numpy as np
import pandas as pd

# Load the model
testingModel = joblib.load("joblibs/gradient_boosting_model.joblib")

# Path to the input file (replace with your actual file path)
file_path = "data/transformedData.csv"

# Read the file into a DataFrame
data = pd.read_csv(file_path)

# Specify the row index and the desired columns (replace with your selections)
row_index = 3  # Select the row you want (e.g., 0 for the first row)
desired_columns = np.array(['fight_duration_lastrnd', 'height_x', 'n_draw_x', 'n_loss_x', 'n_win_x', 'reach_x', 'sig_str_abs_pM_x', 'sig_str_def_pct_x', 'sig_str_land_pM_x', 'sig_str_land_pct_x', 'sub_avg_x', 'td_avg_x', 'td_def_pct_x', 'td_land_pct_x', 'height_y', 'n_draw_y', 'n_loss_y', 'n_win_y', 'reach_y', 'sig_str_abs_pM_y', 'sig_str_def_pct_y', 'sig_str_land_pM_y', 'sig_str_land_pct_y', 'sub_avg_y', 'td_avg_y', 'td_def_pct_y', 'td_land_pct_y', 'win_percentage_x', 'win_percentage_y', 'reach_advantage', 'total_fights_x', 'total_fights_y', 'finishing_rate_x', 'finishing_rate_y', 'age_x', 'age_y', 'age_difference', 'decision_method_1', 'decision_method_2', 'decision_method_3', 'decision_method_4', 'decision_method_5', 'decision_method_6', 'decision_method_7', 'decision_method_8', 'stance_x_1', 'stance_x_2', 'stance_x_3', 'stance_y_1', 'stance_y_2', 'stance_y_3', 'time_format_1', 'time_format_2', 'time_format_3', 'weight_class_1', 'weight_class_2', 'weight_class_3', 'weight_class_4', 'weight_class_5', 'weight_class_6', 'weight_class_7', 'weight_class_8', 'weight_class_9', 'weight_class_10', 'weight_class_11', 'weight_class_12', 'weight_class_13', 'weight_class_14', 'weight_class_15', 'weight_class_16', 'weight_class_17', 'weight_class_18', 'weight_class_19', 'weight_class_20', 'weight_class_21', 'weight_class_22', 'weight_class_23', 'weight_class_24', 'weight_class_25', 'weight_class_26', 'weight_class_27', 'weight_class_28', 'weight_class_29', 'weight_class_30', 'weight_class_31', 'weight_class_32', 'weight_class_33', 'weight_class_34', 'weight_class_35', 'weight_class_36', 'weight_class_37', 'weight_class_38', 'weight_class_39', 'weight_class_40', 'weight_class_41', 'weight_class_42', 'weight_class_43', 'weight_class_44', 'weight_class_45', 'weight_class_46', 'weight_class_47', 'weight_class_48', 'weight_class_49', 'weight_class_50', 'weight_class_51', 'weight_class_52', 'weight_class_53', 'weight_class_54', 'weight_class_55', 'weight_class_56', 'weight_class_57', 'weight_class_58', 'weight_class_59', 'weight_class_60', 'weight_class_61'])

# Extract the selected row and columns
selected_row = data.loc[row_index, desired_columns]

# Convert the selected row into a NumPy array
input_features = np.array(selected_row).astype(float)  # Ensure all values are numerical

# Reshape the input for the model
reshaped_input = input_features.reshape(1, -1)

# Make predictions
predictions = testingModel.predict(reshaped_input)

print("Prediction:", predictions)
print("done")



FileNotFoundError: [Errno 2] No such file or directory: 'data/TransformedData.csv'